In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('oil_price.csv')
df

In [ ]:
df['date'] = pd.to_datetime(df['date'], utc=True)
df

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day'].max()

In [ ]:
df['cos_year'] = np.cos(2 * np.pi * df['year'] / 2024)  # Assuming a year has 2024 days
df['sin_year'] = np.sin(2 * np.pi * df['year'] / 2024)

df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)

df['cos_day'] = np.cos(2 * np.pi * df['day'] / 31)  # Assuming a month has 31 days
df['sin_day'] = np.sin(2 * np.pi * df['day'] / 31)

In [ ]:
# split the data into train and test
from sklearn.model_selection import train_test_split

X = df[['cos_year', 'sin_year', 'cos_month', 'sin_month', 'cos_day', 'sin_day']]
y = df['close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'regression',
    'metric': 'mse',  # Mean Squared Error
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'seed': 42
}


# Define the LightGBM regressor
reg = lgb.LGBMRegressor(**params)

In [ ]:
X_train

In [ ]:
reg.fit(X_train, y_train, verbose=1)

In [ ]:
y_pred = reg.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 
 

In [ ]:
import dill as pickle
with open(f'predictive.pkl', 'wb') as handle:
    pickle.dump(reg, handle, protocol=pickle.HIGHEST_PROTOCOL, recurse=True)

In [ ]:
start_date = pd.to_datetime('today').normalize()
end_date = start_date + pd.DateOffset(days=60)

date_range = pd.date_range(start=start_date, end=end_date, freq='D')
df_next = pd.DataFrame(date_range)


In [ ]:
df_next

In [ ]:
df_next['year'] = df_next[0].dt.year
df_next['month'] = df_next[0].dt.month
df_next['day'] = df_next[0].dt.day

df_next['cos_year'] = np.cos(2 * np.pi * df_next['year'] / 2024)
df_next['sin_year'] = np.sin(2 * np.pi * df_next['year'] / 2024)

df_next['cos_month'] = np.cos(2 * np.pi * df_next['month'] / 12)
df_next['sin_month'] = np.sin(2 * np.pi * df_next['month'] / 12)

df_next['cos_day'] = np.cos(2 * np.pi * df_next['day'] / 31)  # Assuming a month has 31 days
df_next['sin_day'] = np.sin(2 * np.pi * df_next['day'] / 31)

In [ ]:
df_next

In [ ]:
y_pred_next = reg.predict(df_next[X_train.columns])
y_pred_next